In [1]:
%matplotlib inline

In [2]:
from rbm_class.rbm import RBM 
from rbm_class.rbm import save_images 
from rbm_class.dataset import DataSet

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import random
import gzip, struct

# DataSet

In [4]:
def test_DataSet():
    train_dir = {
        'X': './mnist/train-images-idx3-ubyte.gz', 
        'Y': './mnist/train-labels-idx1-ubyte.gz'
    }
    test_dir = {
        'X': './mnist/t10k-images-idx3-ubyte.gz', 
        'Y': './mnist/t10k-labels-idx1-ubyte.gz'
    }
    mnist = DataSet(test_dir, 2)
    print('batch index: %d' % mnist.batch_index)
    X, Y = mnist.next_batch()
    print('batch index: %d' % mnist.batch_index)
    print('X:')
    print(X)
    print('Y:')
    print(Y)

In [5]:
test_DataSet()

batch index: 0
batch index: 1
X:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Y:
[8 7]


In [6]:
train_dir = {
    'X': './mnist/train-images-idx3-ubyte.gz', 
    'Y': './mnist/train-labels-idx1-ubyte.gz'
}

In [7]:
test_dir = {
    'X': './mnist/t10k-images-idx3-ubyte.gz', 
    'Y': './mnist/t10k-labels-idx1-ubyte.gz'
}

In [8]:
logs_dir = './logs'
samples_dir = './samples'

# Model

In [9]:
rbm = RBM()

![](markov_chain.png)

# Data

In [10]:
train_data = DataSet(data_dir=train_dir, batch_size=128, one_hot=True)
test_data = DataSet(data_dir=test_dir, batch_size=128, one_hot=True)

# Train

In [11]:
#x = tf.placeholder(tf.float32, shape=[None, 784])
#noise_x, _ = test_data.sample_batch()
#noise_x = tf.random_normal([test_data.batch_size, 784])
#rbm = RBM()
#step = rbm.learn(x)
#sampler = rbm.sampler(x)
#pl = rbm.pseudo_likelihood(x)
#saver = tf.train.Saver()

In [12]:
def train(train_data,epoches,rbm,model_name):
    x = tf.placeholder(tf.float32, shape=[None, 784])
    step = rbm.learn(x)
    pl = rbm.pseudo_likelihood(x)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        mean_cost = []
        epoch = 1
        for i in range(epoches):
            batch_x, _ = train_data.next_batch()
            sess.run(step, feed_dict = {x: np.floor(batch_x+0.5), rbm.lr: 0.1})
            cost = sess.run(pl, feed_dict = {x: batch_x})
            mean_cost.append(cost)
            # save model
            if i%5000 == 0:
                checkpoint_path = os.path.join(logs_dir, model_name)
                saver.save(sess, checkpoint_path, global_step = epoch + 1)
                print('Saved Model.')
                print('Cost %g' % (np.mean(mean_cost)))
            if i %200 == 0:
                print('Epoch %d Cost %g' % (epoch, np.mean(mean_cost)))
            epoch +=1
        checkpoint_path = os.path.join(logs_dir, model_name)
        saver.save(sess, checkpoint_path, global_step = epoch + 1)
        print('Saved Model.')
        print('Cost %g' % (np.mean(mean_cost)))

In [13]:
Model_Name = 'RBM_Long_Trainning'
train(train_data,20000,rbm,Model_Name)
#if scipy image save does not work follow the instructions proposed here https://github.com/oduerr/dl_tutorial/issues/3

Saved Model.
Cost -16.8793
Epoch 1 Cost -16.8793
Epoch 201 Cost -3.78703
Epoch 401 Cost -2.69382
Epoch 601 Cost -2.00424
Epoch 801 Cost -1.6016
Epoch 1001 Cost -1.34218
Epoch 1201 Cost -1.15952
Epoch 1401 Cost -1.02245
Epoch 1601 Cost -0.915237
Epoch 1801 Cost -0.829438
Epoch 2001 Cost -0.757805
Epoch 2201 Cost -0.698612
Epoch 2401 Cost -0.648156
Epoch 2601 Cost -0.605147
Epoch 2801 Cost -0.567646
Epoch 3001 Cost -0.534898
Epoch 3201 Cost -0.505918
Epoch 3401 Cost -0.480094
Epoch 3601 Cost -0.456945
Epoch 3801 Cost -0.435872
Epoch 4001 Cost -0.416797
Epoch 4201 Cost -0.399528
Epoch 4401 Cost -0.383578
Epoch 4601 Cost -0.369129
Epoch 4801 Cost -0.355647
Saved Model.
Cost -0.343172
Epoch 5001 Cost -0.343172
Epoch 5201 Cost nan
Epoch 5401 Cost nan
Epoch 5601 Cost nan
Epoch 5801 Cost nan
Epoch 6001 Cost nan
Epoch 6201 Cost nan
Epoch 6401 Cost nan
Epoch 6601 Cost nan
Epoch 6801 Cost nan
Epoch 7001 Cost nan
Epoch 7201 Cost nan
Epoch 7401 Cost nan
Epoch 7601 Cost nan
Epoch 7801 Cost nan
Epoch

Sample images from the distribution 

In [11]:
def Sample_Images(rbm,model_name):
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess,model_name)
        x = tf.placeholder(tf.float32, shape=[None, 784])
        train_data_1 = DataSet(data_dir=train_dir,batch_size=64,one_hot=True)
        noise_x,_ = train_data_1.next_batch()
        sampler = rbm.sampler(x)
        print('Test')
        samples = sess.run(sampler, feed_dict = {x: np.floor(noise_x+0.5), rbm.beta:1.0})
        samples = samples.reshape([train_data_1.batch_size, 28, 28])
        save_images(samples, [8, 8], os.path.join(samples_dir, 'RBM_Random_Weights.png'))
        print('Saved samples.')

In [12]:
Sample_Images(rbm,'./logs/RBM_Random_Weights-2')

INFO:tensorflow:Restoring parameters from ./logs/RBM_Random_Weights-2
Test
Saved samples.


## Verification of XTF Theorem 

In [ ]:
import csv
#SAVE THE DATA
def Save_Data(data, name):
    ''' Save a list data in a .csv file '''
    out = csv.writer(open(name,'a'),delimiter=',', quoting=csv.QUOTE_ALL)
    for i in data:
        out.writerow([i])
        
#save_data(P_DE, 'calor.csv')

In [ ]:
#RESTORE SAVE DATA
def Restore_data(file_name):
    ''' Restore the information .csv file in a python list '''
    data=[]
    with open(file_name, 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            data.append(float(list(row)[0]))
        #reader = csv.reader(f)
        #restored_list = list(reader)[0]
        #data = [float(i) for i in restored_list]
        return data

#heat = Restore_data('Calor.csv')

In [ ]:
def Test_XFT_Theorem(rbm,beta2,amount_samples,gibbs_steps,model_name,amount_data):
    with tf.Session() as sess:
        x = tf.placeholder(tf.float32, shape=[None, rbm.n_visible])
        y = tf.placeholder(tf.float32, shape=[None, rbm.n_hidden])
        saver = tf.train.Saver()
        saver.restore(sess,model_name)
        gibbs = rbm.Gibbs_Sampling(x)
        energy = rbm.energy(x,y)
        energy_variations = []
        for i in range(amount_samples//amount_data):
            initial_state = np.random.rand(amount_data,rbm.n_visible)
            #print(i)
            rbm.k=gibbs_steps
            rbm.beta=1.0
            visible_1,hidden_1 = sess.run(gibbs, feed_dict = {x:np.floor(initial_state+0.5)})
            energy_1 = sess.run(energy,feed_dict = {x:visible_1,y:hidden_1})
            rbm.k=1
            rbm.beta=beta2
            visible_2,hidden_2 = sess.run(gibbs,feed_dict={x:visible_1})
            energy_2 = sess.run(energy,feed_dict = {x:visible_2,y:hidden_2})
            delta_energy = energy_2 - energy_1
            energy_substep = []
            for m in delta_energy:
                energy_variations.append(float(m))
                energy_substep.append(float(m))
                print(m)
            name_file = './Data/Delta_Energy_Temperature_'+str(1.0/beta2)+'.csv'
            Save_Data(energy_substep,name_file)
            
    return energy_variations

In [ ]:
energy_variations = Test_XFT_Theorem(rbm,1.1,1000,150,'./logs/RBM_trainning-2002',100)

In [ ]:
Amount_Samples = 2000000
Gibbs_Step = 150
Amount_Data = 100
Model_Name = './logs/RBM_Long_Trainning-20002'

In [ ]:
Temperature = 1.1
Beta = 1.0/Temperature
energy_variations_1 = Test_XFT_Theorem(rbm,Beta,Amount_Samples,Gibbs_Steps,Model_Name,Amount_Data)

In [ ]:
Temperature = 1.2
Beta = 1.0/Temperature
energy_variations_2 = Test_XFT_Theorem(rbm,Beta,Amount_Samples,Gibbs_Steps,Model_Name,Amount_Data)

In [ ]:
Temperature = 0.9
Beta = 1.0/Temperature
energy_variations_3 = Test_XFT_Theorem(rbm,Beta,Amount_Samples,Gibbs_Steps,Model_Name,Amount_Data)    

In [ ]:
Temperature = 0.8
Beta = 1.0/Temperature
energy_variations_4 = Test_XFT_Theorem(rbm,Beta,Amount_Samples,Gibbs_Steps,Model_Name,Amount_Data)

# Reference

1. [GitHub: tensorflow-rbm](https://github.com/meownoid/tensorfow-rbm)
2. [Theano: RBM](http://deeplearning.net/tutorial/rbm.html#rbm)
3. [Stackoverflow: RBM implementation](http://stackoverflow.com/questions/34760981/rbm-implementation-with-tensorflow/35446666#35446666)